In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
import copy, math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/insurance/insurance.csv")
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
for c in ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']:
    print("---- %s ---" % c)
    print(df[c].value_counts())

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
sns.heatmap(df.corr(),cmap='coolwarm',ax=ax,annot=True,linewidths=2)
plt.show()

In [ ]:
num_cols = ['age', 'bmi', 'children', 'charges']
plt.figure(figsize=(18,9))
df[num_cols].boxplot()
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder 
from scipy import stats
df_copy = df.copy()
    
le = LabelEncoder()
df_copy['sex'] = le.fit_transform(df['sex'])
df_copy['smoker'] = le.fit_transform(df['smoker'])
df_copy['region'] = le.fit_transform(df['region'])
df_copy['age'] = stats.zscore(df['age'])
df_copy['bmi'] = stats.zscore(df['bmi'])
df_copy

In [ ]:
from sklearn.model_selection import train_test_split
X = df_copy.drop(columns=['charges'])
y = df_copy['charges']
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
x_train, y_train, x_test, y_test = x_train.to_numpy(), y_train.to_numpy(), x_test.to_numpy(), y_test.to_numpy()

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(x_train, y_train)
y_pred = linreg.predict(x_test)
y_predtrain = linreg.predict(x_train)

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_train, y_predtrain)
print("R-squared score:", r2)

In [ ]:
def cost(X, y, w, b):
    m = X.shape[0]
    cost = 0.
    for i in range(m):
        cost += (np.dot(X[i], w) + b - y[i])**2
    return cost/(2 * m)

In [ ]:
def gradient(X, y, w, b):
    m,n = X.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + (err * X[i, j])[0]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

In [ ]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    J_history = []
    w = copy.deepcopy(w_in)
    b = b_in
    
    for i in range(num_iters):

        dj_db,dj_dw = gradient_function(X, y, w, b)
        w = w - alpha * dj_dw   
        b = b - alpha * dj_db 
      
        if i<100000:  
            J_history.append( cost_function(X, y, w, b))

        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost:", J_history[-1])
        
    return w, b, J_history
    

In [ ]:
from sklearn.model_selection import train_test_split
X = df_copy.drop(columns=['charges'])
y = df_copy['charges']
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.7)
x_train, y_train, x_test, y_test = x_train.to_numpy(), y_train.to_numpy(), x_test.to_numpy(), y_test.to_numpy()

In [ ]:
w_init = 0.01 * (np.random.rand(6).reshape(-1,1) - 0.5)
initial_w = np.zeros_like(w_init)
initial_b = 0.
# some gradient descent settings
iterations = 10000
alpha = 0.0001
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(x_train, y_train, initial_w, initial_b,
                                                    cost, gradient, 
                                                    alpha, iterations)